In [ ]:
#Prepare Data From CSV Recording

##Load File

'Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).'

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.'

'The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.' 

'The new trimmed file is written to a directory (/trim_output), for further manipulation and loading into the model.'


In [ ]:
import numpy as np 
import pandas as pd

recording_path = "../recordings/"
output_path = "../trim_output/"
file_name = input("Input File Name")

#Read in CSV

try:
    dataframe = pd.read_csv(recording_path + file_name + ".csv")
except:
    print("Error Reading File: Check Spelling and Try Again")

# Seperate each tracker to seperate dataframe
HMD = dataframe.iloc[:, 1:4]
controller_1 = dataframe.iloc[:, 7:10]
controller_2 = dataframe.iloc[:, 37:40]
left_foot = dataframe.iloc[:, 67:70]
right_foot = dataframe.iloc[:, 73:76]
waist = dataframe.iloc[:, 79:82]

# Join all trackers together
joined = pd.concat([HMD, controller_1, controller_2,
                   waist, left_foot, right_foot], axis=1)

# set new column headers
joined.columns = [
    "head_x",
    "head_y",
    "head_z",
    "r_controller_x",
    "r_controller_y",
    "r_controller_1_z",
    "l_controller_x",
    "l_controller_y",
    "l_controller_z",
    "waist_x",
    "waist_y",
    "waist_z",
    "r_foot_x",
    "r_foot_y",
    "r_foot_z",
    "l_foot_x",
    "l_foot_y",
    "l_foot_z"
]


# output to new csv
output_file = output_path + file_name + "_trimmed.csv"
joined.to_csv(output_file, index=False)

print(file_name + " output to " + output_path)


In [ ]:
#Data Normalization

## Data Scaling

'The new CSV is loaded into memory, chosen through an input prompt'
'The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).'
'These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100'

In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"

file_name = input("Input File Name")

#read in formatted CSV
try:
    dataframe = pd.read_csv(output_path + file_name + ".csv")
    print("Dataframe created")
except:
    print("Error Reading File")


#Show shape of the dataframe
dataframe.describe()

x_train_df = dataframe.iloc[:, 0:9]
y_train_df = dataframe.iloc[:, 9:18]
#display(x_train_df)
#display(y_train_df)

#Load data into Numpy array
x_train = np.array(x_train_df)
y_train = np.array(y_train_df)



#Divide values in array by 100
x_train_normalized = np.divide(x_train, 100)
y_train_normalized = np.divide(y_train, 100)

x_train_normalized

#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)


X = np.reshape(x_train_normalized, (len(x_train_normalized), seq_length, 1))



Dataframe created


array([[ 0.08953532,  1.59178848, -0.17578089, ...,  0.03788082,
         0.80374146,  0.09096572],
       [-0.25591087,  1.58697922,  0.33253227, ..., -0.13299416,
         0.79182411,  0.42571774],
       [-0.09145373,  1.5878537 ,  0.55729786, ...,  0.12953456,
         0.8707579 ,  0.54476894],
       ...,
       [-0.23291313,  1.58764175,  0.39430336, ..., -0.08221909,
         0.83395187,  0.52449009],
       [-0.33366257,  1.61500504, -0.13528797, ..., -0.5369175 ,
         0.9034877 , -0.04983424],
       [-0.08755193,  1.58678925, -0.16763311, ..., -0.25641184,
         0.85932747, -0.00966894]])

array([[-0.26865009,  1.58726578,  0.29906637, ..., -0.14812251,
         0.77162163,  0.36663391],
       [ 0.49014988,  1.61212372,  0.06012411, ...,  0.36656395,
         0.87946587, -0.16293947],
       [-0.28961931,  1.58733063,  0.2441571 , ..., -0.15265855,
         0.74716797,  0.26382154],
       ...,
       [ 0.45292736,  1.58303238,  0.04115045, ...,  0.31590321,
         0.79069298, -0.03898935],
       [-0.26975645,  1.59310135, -0.21043425, ..., -0.44748478,
         0.89316277, -0.02274265],
       [ 0.48439808,  1.60975769,  0.02213422, ...,  0.34453053,
         0.89163879, -0.19003639]])

array([[ 0.09357481,  1.01723122, -0.18644894, ..., -0.08056785,
         0.12744571, -0.0297064 ],
       [-0.33355206,  0.98410858,  0.32017376, ..., -0.35795235,
         0.09746568,  0.17695101],
       [-0.10645221,  0.99120728,  0.60327942, ..., -0.14961736,
         0.1388325 ,  0.44850548],
       ...,
       [-0.3023633 ,  0.98279724,  0.39181393, ..., -0.35525925,
         0.09754041,  0.18381891],
       [-0.32639297,  1.00579445, -0.15693558, ..., -0.17783466,
         0.10114307, -0.11393599],
       [-0.06775779,  1.00911591, -0.18631691, ...,  0.00547598,
         0.0982889 , -0.10124354]])

array([[-0.3481485 ,  0.98384544,  0.28281727, ..., -0.35897537,
         0.09767203,  0.1736515 ],
       [ 0.55726334,  0.99712975,  0.02524828, ...,  0.54674141,
         0.10101665,  0.16679667],
       [-0.37034046,  0.98455711,  0.22257441, ..., -0.35870018,
         0.09900968,  0.1705999 ],
       ...,
       [ 0.520989  ,  1.00314018,  0.01301736, ...,  0.53396984,
         0.09212357,  0.0915747 ],
       [-0.25951385,  0.9939151 , -0.20266769, ..., -0.00253506,
         0.09631337, -0.10413281],
       [ 0.54222069,  0.99996735, -0.00909552, ...,  0.54647499,
         0.10011427,  0.16431543]])

#Model Creation and Training

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
model = keras.Sequential()

model.add(
    layers.Bidirectional(layers.LSTM(
        64, return_sequences=True), input_shape=(5, 10))
)
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(10))

model.summary()
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 5, 128)           38400     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 80,266
Trainable params: 80,266
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (N